In [1]:
!pip install newsapi-python



In [2]:
from newsapi import NewsApiClient
import pandas as pd

# Initialize the client
api_key = 'c44235de961a45f1ac4f933dddaecb55'  # Replace 'YOUR_API_TOKEN' with your actual API token
newsapi = NewsApiClient(api_key=api_key)


In [3]:
top_headlines = newsapi.get_top_headlines(country='us', category='business', page_size=100)


In [4]:
# Let's see the keys in the returned data
print(top_headlines.keys())

# Print the first article to see its structure
print(top_headlines['articles'][0])


dict_keys(['status', 'totalResults', 'articles'])
{'source': {'id': None, 'name': 'CNBC'}, 'author': 'Evelyn Cheng, Clement Tan', 'title': "China's property troubles aren't getting better, intensifying calls for bolder policy help - CNBC", 'description': 'Prospective home buyers are holding back on making purchases, leading to weak sales that compound the urgent need for policymakers to step up support for the industry.', 'url': 'https://www.cnbc.com/2023/08/18/chinas-property-troubles-worsen-ramping-calls-for-bolder-policy-help.html', 'urlToImage': 'https://image.cnbcfm.com/api/v1/image/107287878-1692248610087-gettyimages-1250413357-Rural_Infrastructure_Construction_In_China.jpeg?v=1692323081&w=1920&h=1080', 'publishedAt': '2023-08-18T01:44:00Z', 'content': "Aerial photo shows a rural residential area in Chengdong town of Hai 'an City, East China's Jiangsu Province, April 1, 2023.\r\nChina's real estate troubles are accelerating. Prospective home buyers ar… [+5832 chars]"}


# producer 

In [ ]:
from newsapi import NewsApiClient
import json
from kafka import KafkaProducer

# Initialize the NewsAPI client with your API key.
key = "c44235de961a45f1ac4f933dddaecb55"
newsapi = NewsApiClient(api_key=key)

# Fetch top business headlines from the U.S.
all_articles = newsapi.get_top_headlines(country='us', category='business')

# Initialize the Kafka producer
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Send articles to the "FinalProject" Kafka topic
for article in all_articles['articles']:
    print(article['title'])
    producer.send('FinalProject', json.dumps(article).encode('utf-8'))

# Close the Kafka producer
producer.close()
---------------------------------------------------------------------------------------------------------



# consumer

In [ ]:
from kafka import KafkaConsumer
import csv
import json
from hdfs import InsecureClient

# Kafka Consumer Setup
consumer = KafkaConsumer('FinalProject', bootstrap_servers='localhost:9092')

# Writing to CSV
csv_filename = 'headline.csv'
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Source", "Description"])  # Write header
    
    try:
        for message in consumer:
            article = json.loads(message.value.decode('utf-8'))
            
            title = article.get('title', '')
            source = article.get('source', {}).get('name', '')
            description = article.get('description', '')
            
            writer.writerow([title, source, description])
    except KeyboardInterrupt:
        print("Stopped consuming messages and writing to CSV.")
        pass

# Uploading to HDFS
hadoop_client = InsecureClient('https://newsapi.org/v2/top-headlines?country=us&category=business&apiKey=c44235de961a45f1ac4f933dddaecb55//localhost:50070')

with open(headline.csv, 'rb') as file:
    hadoop_client.upload('/file/headline.csv', file)
    print(f"{headline.csv} uploaded to HDFS.")
